In [1]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
You use the Feynman Technique to explain concepts to your students. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You draw inspiration from Edward Frenkel. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

In [3]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    }
]

In [4]:
llm = OpenAI() # type: ignore

In [5]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

In [6]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [7]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [8]:
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

In [10]:
print(chain.run("explain why the entropy of a black hole is proportional to its surface area"))

Error in on_chain_start callback: 'name'


physics: {'input': 'Explain why the entropy of a black hole is proportional to the surface area of the event horizon.'}
> Finished chain.


The entropy of a black hole is proportional to the surface area of the event horizon because of the Bekenstein-Hawking formula. This formula states that the entropy of a black hole is 1/4 of its event horizon's surface area, expressed in terms of the gravitational constant and the speed of light. This formula implies that the entropy of a black hole is proportional to its surface area, as the surface area is the only factor in the equation. 

In order to explain this in more detail, it is useful to understand the concept of entropy. Entropy is a measure of the amount of disorder of a system and is related to the number of different microscopic arrangements of the particles that make up the system. The entropy of a black hole is related to the number of possible states of the particles that make up the black hole. This is because the particles in a 